In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import datetime
import glob
import gc
import os
import sys
import pandas as pd

#========================================================================
# Args
#========================================================================
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

HOME = os.path.expanduser('~')
sys.path.append(f'{HOME}/kaggle/data_analysis/model')

sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from preprocessing import get_ordinal_mapping
from utils import logger_func
logger=logger_func()

from dimensionality_reduction import go_bhtsne, UMAP

2019-02-07 18:46:33,885 utils 396 [INFO]    [logger_func] start 


In [2]:
#========================================================================
# Data Load

win_path = f'../features/4_winner/*.gz'
tmp_path_list = glob.glob(f'../features/5_tmp/*.gz') + glob.glob(f'../features/0_exp/*.gz')

base = utils.read_df_pkl('../input/base_first*')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

win_path_list = glob.glob(win_path) + tmp_path_list
feature_list = utils.parallel_load_data(path_list=win_path_list)

df_feat = pd.concat(feature_list, axis=1)

#  train = pd.concat([base_train, df_feat.iloc[:len(base_train), :]], axis=1)
#  test = pd.concat([base_test, df_feat.iloc[len(base_train):, :].reset_index(drop=True)], axis=1)

#========================================================================

100%|██████████| 3/3 [00:00<00:00, 62.16it/s]


In [10]:
for col in df_feat.columns:
    if col.count('date'):
        val_min = df_feat[col].min()
        df_feat[col].fillna(val_min-100, inplace=True)
    elif col.count('month_diff'):
        val_min = df_feat[col].min()
        df_feat[col].fillna(val_min-10, inplace=True)
    else:
        df_feat[col].fillna(-1, inplace=True)

In [11]:
feat_list = [col for col in df_feat.columns]
emb_list = []

for i in range(1, 3, 1):
    np.random.seed(i)
    tmp_list = np.random.choice(feat_list, 30)
#     embeddeing = go_bhtsne(logger=logger, data=df_feat[tmp_list], D=2)
    embedding = UMAP(logger=logger, data=df_feat[tmp_list], D=2)
    emb_list.append(embedding)

2019-02-07 17:41:47,055 utils 112 [INFO]    [UMAP] UMAP train starttime: 1549528907.0554855 
2019-02-07 17:53:35,003 utils 116 [INFO]    [UMAP] UMAP train end. caliculation time: 707.9480867385864 


NameError: name 'embedding' is not defined

In [4]:
from sklearn.decomposition import PCA
feat_num = 30
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

prefix = f"224_emb_{start_time[4:12]}"
pca = PCA(n_components=0.81,whiten=True)
feature = pca.fit_transform(pd.DataFrame(embedding, columns=['D1', 'D2']))
feature = feature.ravel().astype('float32')
utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_UMAP_feat{feat_num}_seed{i}')

NameError: name 'embedding' is not defined

In [3]:
if feature.ravel().shape[0]>500000:
    feat_1 = feature[:, 0]
    feat_2 = feature[:, 1]
    utils.to_pkl_gzip(obj=feat_1, path=f'../features/1_first_valid/{prefix}_PCA_D1_{str(combi)}')
    utils.to_pkl_gzip(obj=feat_2, path=f'../features/1_first_valid/{prefix}_PCA_D2_{str(combi)}')
else:
    feature = feature.ravel().astype('float32')
    utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_PCA_{str(combi)}')

NameError: name 'feature' is not defined